# Sales Pipeline Summary & Risk Flagging

**User Story**: US2 — Pipeline Summary & Risk Flagging
**Persona**: Sales Manager

This notebook demonstrates how to use the Salesforce AI Assistant to generate
a team pipeline summary with at-risk deals flagged. The assistant analyzes
pipeline data grouped by AE, identifies deals with risk indicators (stalled stage,
overdue close date, no recent activity), and provides actionable recommendations.

## Prerequisites
- `.env` file configured with Salesforce and Azure AI credentials
- `salesforce-crm` MCP server available
- Python packages installed (`pip install -r requirements.txt`)
- Risk thresholds configured in `config/risk_thresholds.yaml`

In [ ]:
# Cell 2: Environment Check
import os
import sys

sys.path.insert(0, os.path.abspath(".."))

from dotenv import load_dotenv

load_dotenv("../.env")

required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise EnvironmentError(f"Missing environment variables: {', '.join(missing)}")

print("✅ Environment configured")
print(f"   Project: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:50]}...")
print(f"   SF Instance: {os.environ['SF_INSTANCE_URL']}")

In [ ]:
# Cell 3: Authenticate with Azure AI Foundry
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)
print("✅ Authenticated with Azure AI Foundry")

In [ ]:
# Cell 4: Configure MCP Connection — salesforce-crm
mcp_connection = project.connections.get("salesforce-crm")
print(f"✅ MCP connection: {mcp_connection.name}")

In [ ]:
# Cell 5: Create Sales Agent
from azure.ai.projects.models import McpToolConnection

with open("../agents/sales/system_prompt.md") as f:
    system_prompt = f.read()

mcp_tool = McpToolConnection(connection=mcp_connection)

agent = project.agents.create_agent(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    name="Sales Pipeline Analyst",
    instructions=system_prompt,
    tools=[mcp_tool],
)
print(f"✅ Agent created: {agent.name} (ID: {agent.id})")

thread = project.agents.threads.create()
print(f"✅ Thread created: {thread.id}")

In [ ]:
# Cell 6: Pipeline Query with Risk Flags
from IPython.display import Markdown, display

message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me my team's pipeline and flag any at-risk deals. "
    "Group the summary by AE and highlight deals that are overdue, "
    "stalled, or have no recent activity.",
)

run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)
print(f"✅ Run completed: {run.status}")

# Display pipeline summary
messages = project.agents.messages.list(thread_id=thread.id)
for msg in messages.data:
    if msg.role == "assistant":
        for content in msg.content:
            if hasattr(content, "text"):
                display(Markdown(content.text.value))
        break

In [ ]:
# Cell 7: Risk-Flagged Deals Deep Dive
message2 = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="For the at-risk deals, what specific actions should each AE take this week?",
)

run2 = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

messages2 = project.agents.messages.list(thread_id=thread.id)
for msg in messages2.data:
    if msg.role == "assistant":
        for content in msg.content:
            if hasattr(content, "text"):
                display(Markdown(content.text.value))
        break

In [ ]:
# Cell 8: AE-Specific Follow-up
AE_NAME = "Sarah"  # Change to a specific AE name

message3 = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What should {AE_NAME} focus on this week?",
)

run3 = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

messages3 = project.agents.messages.list(thread_id=thread.id)
for msg in messages3.data:
    if msg.role == "assistant":
        for content in msg.content:
            if hasattr(content, "text"):
                display(Markdown(content.text.value))
        break

In [ ]:
# Cell 9: Cleanup
project.agents.delete_agent(agent.id)
print(f"✅ Agent deleted: {agent.id}")
print("\n--- Session Complete ---")